In [23]:
import dotenv

dotenv.load_dotenv()

True

In [6]:
from langchain import PromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI

/home/naman/miniconda3/envs/langchain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [9]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

In [12]:
from langchain.tools import BaseTool
from math import pi
from typing import Union


class FlightAndHotelData(BaseTool):
    name: str = "Flight and Hotel data collector"
    description: str = (
        "use this tool when you need the information about the flight and hotel from source to destination"
    )

    def _run(self, source: str, destination: str, from_date: str, to_date: str) -> str:
        return "Data"

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")

In [22]:
import requests
import os

In [24]:
url = "https://tripadvisor16.p.rapidapi.com/api/v1/hotels/searchHotels"

querystring = {
    "geoId": "1584865",
    "checkIn": "2025-02-20",
    "checkOut": "2025-02-25",
    "pageNumber": "1",
    "currencyCode": "INR",
}

headers = {
    "x-rapidapi-key": os.environ.get("RAPID_API_KEY"),
    "x-rapidapi-host": "tripadvisor16.p.rapidapi.com",
}

response = requests.get(url, headers=headers, params=querystring)

In [28]:
for i in response.json()["data"]["data"]:
    print(i["title"])

1. Royal Palace By Rajasthan Maheshwari Bhawan
2. Nirmal Chhaya Nature Resorts
3. Derby Hotel & Restaurant
4. Hotel Raj Palace
5. Hotel indra place
Royal Palace By Rajasthan Maheshwari Bhawan
6. Hotel GSS
7. Tiger's Den Resort
8. Hotel Parinita Shri
9. Hotel Yashraj
10. SPOT ON Krishna Market And Lodge
11. Hotel JK Celebration
12. Hotel Vrindavan
13. Olive
14. Bandhavgarh Tiger Resort
15. Bisman Lodge
16. OYO Flagship Hotel Gurukripa Inn
17. Guru Simran Lodge
18. Nisarg Resort
19. Krishnam Resort
20. Hotel Datt Residency
21. OYO 9305 Hotel Dwarika Inn
22. Oyo Hotel Air Palace
23. Collection O 81290 Hotel Air Palace
24. The Roaring Salvan County
25. Hotel Kadamb Tree
26. Hotel Balaji Pride
27. Hotel Princess 3
28. Ankit Hotel By M Square
29. Shashnoday Jain Bhavan
30. Max Hotel Jabalpur


In [14]:
tools = [load_tools(["serpapi"])[0], FlightAndHotelData()]

In [15]:
tools[0].description = (
    "A search engine. Useful when you need to find information about anything. Use this to find popular attractions, landmarks, and activities at the destination. Input should be a search query."
)

In [16]:
agent = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method="generate",
)

/tmp/ipykernel_44532/2747768743.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(


In [17]:
agent.run("Can you tell me flight from bombay to goa on 24 feb 2025 to 28 feb 2025 ?")

/tmp/ipykernel_44532/3536805.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Can you tell me flight from bombay to goa on 24 feb 2025 to 28 feb 2025 ?")




> Entering new AgentExecutor chain...
Thought:
I can use Flight and Hotel data collector tool to get this information.

Action:
```json
{
  "action": "Flight and Hotel data collector",
  "action_input": {
    "source": "bombay",
    "destination": "goa",
    "from_date": "24 feb 2025",
    "to_date": "28 feb 2025"
  }
}
```
Observation: Data
Thought:Thought:
I can use Flight and Hotel data collector tool to get this information.

Action:
```json
{
  "action": "Flight and Hotel data collector",
  "action_input": {
    "source": "bombay",
    "destination": "goa",
    "from_date": "24 feb 2025",
    "to_date": "28 feb 2025"
  }
}
```
Observation: Data
Thought:Thought:
I can use Flight and Hotel data collector tool to get this information.

Action:
```json
{
  "action": "Flight and Hotel data collector",
  "action_input": {
    "source": "bombay",
    "destination": "goa",
    "from_date": "24 feb 2025",
    "to_date": "28 feb 2025"
  }
}
```
Observation: Data
Thought:Action:
```json
{


'I am sorry, I cannot fulfill this request. The Flight and Hotel data collector tool could not find any information about flights for the specified date.'

In [18]:
a = agent.agent.llm_chain.prompt.messages[0].prompt.template
print(a)

Respond to the human as helpfully and accurately as possible. You have access to the following tools:

Search: A search engine. Useful when you need to find information about anything. Use this to find popular attractions, landmarks, and activities at the destination. Input should be a search query., args: {{'tool_input': {{'type': 'string'}}}}
Flight and Hotel data collector: use this tool when you need the information about the flight and hotel from source to destination, args: {{'source': {{'title': 'Source', 'type': 'string'}}, 'destination': {{'title': 'Destination', 'type': 'string'}}, 'from_date': {{'title': 'From Date', 'type': 'string'}}, 'to_date': {{'title': 'To Date', 'type': 'string'}}}}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or Search, Flight and Hotel data collector

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
  "action": $TOOL_NAME,
  "action_inp

In [19]:
with open("prompt.txt", "r") as f:
    prompt = f.read()

In [20]:
agent.agent.llm_chain.prompt.messages[0].prompt.template = prompt

In [21]:
agent.run(
    "Plan a vacation for me from mumbai to goa starting from 24 feb 2025 to 28 feb 2025"
)



> Entering new AgentExecutor chain...
Thought:
I need to find flight and hotel options from Mumbai to Goa for the specified dates and then suggest popular attractions and activities in Goa.

Action:
```json
{
  "action": "Flight and Hotel data collector",
  "action_input": {
    "source": "Mumbai",
    "destination": "Goa",
    "from_date": "24 Feb 2025",
    "to_date": "28 Feb 2025"
  }
}
```
Observation: Data
Thought:Thought:
I have the flight and hotel data. Now I need to search for popular attractions and activities in Goa.

Action:
```json
{
  "action": "Search",
  "action_input": "popular attractions and activities in Goa"
}
```

Observation: [{'title': 'Basilica of Bom Jesus', 'link': 'https://www.google.com#', 'description': 'Basilica', 'rating': 4.5, 'reviews': 62000, 'price': 'Free', 'thumbnail': 'https://serpapi.com/searches/67569557fb59fe045e380e3a/images/b7a07631ce8d9ecb8efd9241691fe6a3db2492a4b74f7cd1edcf5e4bc8f38b76.jpeg'}, {'title': 'Dudhsagar Falls', 'link': 'https:/

{'vacation_plan': {'destination': 'Goa',
  'dates': '24 Feb 2025 - 28 Feb 2025',
  'flights': 'Unfortunately, I do not have access to real-time data for flights and hotels. Please use online travel agencies or search engines like Kayak, Expedia, MakeMyTrip, or Google Flights to find flight and hotel options for your specified dates.',
  'attractions_and_activities': ['Basilica of Bom Jesus',
   'Dudhsagar Falls',
   'Palolem Beach',
   'Calangute Beach',
   'Agonda Beach',
   'Baga Beach',
   'Arambol Beach',
   'Benaulim Beach',
   'Candolim Beach Road',
   'Aguada Fort',
   'Shree Mangueshi Temple',
   'Varca Beach',
   'Shree Shantadurga Temple',
   'Reis Magos Fort',
   'Se Cathedral',
   'Immaculate Conception Church',
   'Morjim Beach Road',
   'Chapora Fort',
   'Cabo de Rama Fort']}}

In [6]:
prompt = """You are a travel planning assistant. When a user provides travel dates and a destination,
your role is to gather information about available flight options for the specified dates and suggest popular attractions,
landmarks, and activities at the destination. Provide clear and organized details on flights, including options based on convenience and budget, and 
highlight must-visit places that enhance the travel experience.

Travel Dates: {dates}
Source: {source}
Destination: {destination}
Answer:"""

prompt_template = PromptTemplate.from_template(prompt)
print(prompt_template)

input_variables=['dates', 'destination', 'source'] template='You are a travel planning assistant. When a user provides travel dates and a destination,\nyour role is to gather information about available flight options for the specified dates and suggest popular attractions,\nlandmarks, and activities at the destination. Provide clear and organized details on flights, including options based on convenience and budget, and \nhighlight must-visit places that enhance the travel experience.\n\nTravel Dates: {dates}\nSource: {source}\nDestination: {destination}\nAnswer:'


* https://www.pinecone.io/learn/series/langchain/langchain-tools/
* https://www.kaggle.com/code/ilhansevval/build-custom-tools-using-agent-in-langchain